In [6]:
import pandas as pd
from selenium import webdriver
from openpyxl import Workbook
from openpyxl.styles import Alignment, PatternFill, Font, Side, Border, numbers

invoice_table_header_list = [ #컬럼 넘버(알파벳), 컬럼명, 컬럼셀 너비 순서
    ["A", "고객사", 20],
    ["B", "서비스 일자", 12],
    ["C", "서비스 내역", 40],
    ["D", "서비스 기사", 12],
    ["E", "유상/무상", 10],
    ["F", "공급가액(원)", 15],
    ["G", "부가세(원)", 15],
    ["H", "합계(원)", 15]
]

def makeInvoiceWorkbook():
    wb = Workbook() #워크북 객체 생성
    ws = wb["Sheet"] #자동으로 생성된 첫 번째 시트 선택
    ws.title = "서비스 청구서" #선택된 첫 번째 시트의 이름 변경
    
    ws["A2"] = "(주)카피맨 렌탈 복사기 서비스 비용 청구서"
    ws["A4"] = "※서비스 기간 : 2021년 10월"
    
    #청구서 표의 헤더 입력
    for header in invoice_table_header_list:
        ws[header[0] + "5"] = header[1]

    ws["A8"] = "위 금액을 청구합니다."
    ws["A9"] = "2021년 10월 29일"
    ws["A10"] = "(주)카피맨 대표이사 김카피(인)"
    
    return wb

def stylizeInvoiceForm(ws, item_start_row_no, item_row_no):
    #서비스 이력표 테두리 서식과 적용 영역을 미리 지정
    side = Side(color="CCCCCC", border_style="medium")
    border_style = Border(left = side, right = side, top = side, bottom = side)

    #1. 서비스 이력표 영역을 반복문으로 접근하며 정렬/폰트서식 설정
    for header in invoice_table_header_list:
        ws.column_dimensions[header[0]].width = header[2] #셀 너비 설정
        
        #1-1) 헤더 행 서식 설정
        ws[header[0] + "5"].font = Font(sz="12", bold=True)
        ws[header[0] + "5"].fill = PatternFill(patternType="solid", start_color="ffd663")
        ws[header[0] + "5"].border = border_style
        ws[header[0] + "5"].alignment = Alignment(horizontal = "center", vertical = "center")
        
        #1-2) 내용 행 서식 설정. 내용 시작행부터 종료행까지 반복문으로 접근하여 설정
        for row_no in range(item_start_row_no, item_row_no+1):
            ws["{}{}".format(header[0], row_no)].font = Font(sz="11", bold=False)
            ws["{}{}".format(header[0], row_no)].border = border_style
            if header[0] in ["A", "B", "C", "D", "E"]: #A~E 열은 중앙 정렬
                ws["{}{}".format(header[0], row_no)].alignment = Alignment(horizontal = "center", vertical = "center")
            else: #이후 열은 숫자(금액)이므로 오른쪽 정렬, 세자리수마다 콤마 입력
                ws["{}{}".format(header[0], row_no)].alignment = Alignment(horizontal = "right", vertical = "center")
                ws["{}{}".format(header[0], row_no)].number_format = numbers.BUILTIN_FORMATS[37]

        #1-3) 총계 행 서식 설정
        ws["{}{}".format(header[0], item_row_no)].font = Font(bold=True)
        ws["{}{}".format(header[0], item_row_no)].fill = PatternFill(patternType="solid", start_color="eeeeee")

    #2. 셀 병합 처리
    ws.merge_cells("A2:H2")
    ws.merge_cells("A4:H4")
    ws.merge_cells("B{}:E{}".format(item_row_no, item_row_no))
    ws.merge_cells("A{}:H{}".format(item_row_no+2, item_row_no+2))
    ws.merge_cells("A{}:H{}".format(item_row_no+3, item_row_no+3))
    ws.merge_cells("A{}:H{}".format(item_row_no+4, item_row_no+4))
    
    #3. 서비스 이력 표 이외 부분의 정렬/폰트서식 처리
    ws["A2"].alignment = Alignment(horizontal = "center", vertical = "center")
    ws["A2"].font = Font(sz="20", bold=True)
    
    ws["A4"].alignment = Alignment(horizontal = "right", vertical = "center")
    
    ws["A{}".format(item_row_no+2)].alignment = Alignment(horizontal = "center", vertical = "center")
    ws["A{}".format(item_row_no+2)].font = Font(sz="16", bold=True)
    
    ws["A{}".format(item_row_no+3)].alignment = Alignment(horizontal = "center", vertical = "center")
    ws["A{}".format(item_row_no+3)].font = Font(sz="12", bold=True)
    
    ws["A{}".format(item_row_no+4)].alignment = Alignment(horizontal = "right", vertical = "center")
    ws["A{}".format(item_row_no+4)].font = Font(sz="14", bold=True)

    #4. 행 높이 설정
    ws.row_dimensions[2].height = 40
    ws.row_dimensions[4].height = 20
    ws.row_dimensions[5].height = 25
    ws.row_dimensions[item_row_no+2].height = 40
    ws.row_dimensions[item_row_no+3].height = 20
    ws.row_dimensions[item_row_no+4].height = 40

    for row_no in range(item_start_row_no, item_row_no+1):
        ws.row_dimensions[row_no].height = 25
    
    #서식 처리가 완료된 워크시트 리턴
    return ws

#sheet_name을 None으로 설정하면 모든 시트를 로드
#각 시트를 df로 로드하고, df들을 하나의 딕셔너리 자료형으로 묶어 리턴해 줌
df_src = pd.read_excel("서비스 이력.xlsx", sheet_name = None)

#concat() 메서드를 사용하여 8개 시트 데이터를 하나의 df로 병합
df = pd.concat(df_src, ignore_index=True)
customer_list = df["고객사명"].unique()

for customer in customer_list:
    wb = makeInvoiceWorkbook()
    ws = wb["서비스 청구서"]
    
    #반복문에서 현재 접근하고 있는 고객사 이름으로 필터링
    this_customer_items = df.loc[df["고객사명"] == customer] 

    #방문일자, 기사명 순으로 정렬될 수 있도록 함
    this_customer_items = this_customer_items.sort_values(by=["방문일자", "기사"])

    item_row_no = 6 #서비스 이력 내용이 입력되는 행의 번호
    for idx in range(len(this_customer_items)): #서비스 이력 개수만큼 반복
        ws.insert_rows(item_row_no) #서비스 이력이 들어갈 곳에 새로운 행 삽입
        #고객사, 서비스 일자, 서비스 내역 등 데이터를 컬럼에 맞게 입력
        ws["A{}".format(item_row_no)] = this_customer_items.iloc[idx, 1]
        ws["B{}".format(item_row_no)] = this_customer_items.iloc[idx, 0]
        ws["C{}".format(item_row_no)] = this_customer_items.iloc[idx, 3]
        ws["D{}".format(item_row_no)] = this_customer_items.iloc[idx, 2]
        ws["E{}".format(item_row_no)] = this_customer_items.iloc[idx, 4]
        ws["F{}".format(item_row_no)] = this_customer_items.iloc[idx, 5]
        ws["G{}".format(item_row_no)] = this_customer_items.iloc[idx, 6]
        ws["H{}".format(item_row_no)] = this_customer_items.iloc[idx, 7]
        item_row_no += 1 #다음 행에 서비스 이력이 입력될 수 있도록 변수값을 1 증가시킴
    
    #item_row_no : 서비스 이력이 입력된 행의 다음 행 번호가 저장되어 있음
    ws["A{}".format(item_row_no)] = "총계"
    ws["B{}".format(item_row_no)] = '=COUNTA(C{}:C{})&"건"'.format(6, item_row_no-1)
    ws["F{}".format(item_row_no)] = "=SUM(F{}:F{})".format(6, item_row_no-1)
    ws["G{}".format(item_row_no)] = "=SUM(G{}:G{})".format(6, item_row_no-1)
    ws["H{}".format(item_row_no)] = "=SUM(H{}:H{})".format(6, item_row_no-1)
    
    ws = stylizeInvoiceForm(ws, 6, item_row_no)
    
    wb.save("서비스 청구서_{}.xlsx".format(customer)) #엑셀파일로 저장